**Criação de um índice de criminalidade para análises**



O índice é calculado por ano. Para isso, criarei uma tabela para cada ano, em cada coluna referente a um crime está todas as ocorrências de tal crime no ano, e cada linha um município da região administrativa de campinas.

Para o cálculo do índice, seguirei a fórmula desse artigo (com algumas adaptações pois o número de crime analisados pelos autores foi menor): [Definition of Strategies for Crime Prevention and Combat Using Fuzzy Clustering and Formal Concept Analysis](https://www.researchgate.net/publication/320024255_Definition_of_Strategies_for_Crime_Prevention_and_Combat_Using_Fuzzy_Clustering_and_Formal_Concept_Analysis).

A tabela orignal, os pesos dados para cada crime e a fórmula orinal estão abaixo:

| Crime                       | Factor | Weight                                                                                          |
|-----------------------------|--------|-------------------------------------------------------------------------------------------------|
| Theft (victim not present)   | 1      | Theft Weight = (theft dangerousness factor) × (number of theft occurrences)                      |
| Robbery                     | 2      | Robbery Weight = (robbery dangerousness factor) × (number of robbery occurrences)                |
| Homicide                    | 3      | Homicide Weight = (homicide dangerousness factor) × (number of homicide occurrences)             |
| Drug Traffic                | 4      | Drug Traffic Weight = (traffic dangerousness factor) × (number of traffic occurrences)           |

*Table. Calculation of the Danger Factor for each crime type.*

**Fórmula:**


$\text{Danger Level} = \frac{\text{Theft.W} + \text{Robbery.W} + \text{Homicide.W} + \text{Drug.Traffic.W}}{\text{Total.Weight}}$


Como consta no artigo, os pesos para cada crime foram atribuídos por especialistas. "The weighting
of each type of crime was defined using common sense and the opinion of experts"

No nosso trabalho, analisaremos os seguintes crimes:

|Crimes                    |
|---------------------------------|
|Furto de veículo|          
|Furtos outros|             
|Homicídio culposo outros|          
|Homicídio culposo por acidente de trânsito|        
|Latrocínio|         |
|Lesão corporal culposa outras|        
|Lesão corporal culposa por acidente de trânsito|        
|Lesão corporal dolosa|       |
|Homicídio doloso (nº de vítimas)|              
|Tentativa de homicídio|                   
|Total Estupro|                      
|Total de roubos|                     


Como alguns dos crimes que analisaremos não constam na lista de crimes do artigo - e vice-versa -, e alguns deles possuem algumas variações no nossos dados, como homicídio, por exemplo, que nos dados do artigo é unificado, nos nossos dados possui uma separação como visto na tabela acima. Para lidar com isso, faremos algumas modificações nos pesos dados a cada crime.

Para isso, aumentaremos a escala de 1-4 para 1-6. E as explicações para cada peso seguem abaixo de cada crime e seu respectivo crime.

**Pesos mantidos:**

* Furto de veículo (peso 1)
* Furtos outros (peso 1)
* Total de roubos (peso 2)

Para esses crimes, manteremos os pesos dados no artigo, pois acreditamos não haver muita diferença entre o tipo de crime.

**Pesos atualizados por detalhamento de crime**


* Homicídio culposo por acidente de trânsito (peso 4.5)
    * Por envolver conduta negligente, imperita ou imprudente no trânsito, que podem ser causada por outros crimes, como dirigir sob efeito de álcool, acrescentamos 0.5 para ponderar a possível ocorrência - concomitante - de outro crime. Além disso, a pena é de 2 a 4 anos, enquanto a pena de homicídio culposo, de 1 a 3.

* Homicídio culposo outros (peso 4)
    * Como não há a especificação da ocorrência de outro crime concomitantemente, deixaremos 0.5 abaixo do homicídio culposo por acidente
* Homicídio doloso (peso 5)
    * Explicação

**Pesos criados para crimes novos**

* Latrocínio (peso 6)
* Lesão corporal culposa outras (peso 2.5)
* Lesão corporal culposa por acidente de trânsito (peso 3)   
* Lesão corporal dolosa (peso 3.5)
* Total Estupro (peso 5)
* Tentativa de homicídio (peso 4.5)
    * Por não haver consumação do ato, retiramos 0.5 em relação ao homicídio doloso

Com isso, nossa tabela atualizada com pesos ficou assim:

**Tabela atualizada:**

|Crimes                    |Pesos  |
|---------------------------------|-------------|
|Furto de veículo|       1   |
|Furtos outros|             1 |
|Total de roubos|   2|
|Lesão corporal culposa outras|       2.5 |
|Lesão corporal culposa por acidente de trânsito|    3    |
|Lesão corporal dolosa|       3.5|
|Homicídio culposo outros|        4  |
|Homicídio culposo por acidente de trânsito|     4.5   |
|Tentativa de homicídio|           4.5      |
|Homicídio doloso (nº de vítimas)|     5   |     
|Total Estupro|            5        |  
|Latrocínio|      6   |


**Fórmula atualizada:**

$\text{Índice de Criminalidade} = \frac{\sum_{i=1}^{12} \frac{C_i}{população}  \times P_i}{\sum_{i=1}^{12} P_i}$

Para deixar o índice mais consistente optamos usar crime per capita ao ínves de ocorrências do crime, para evitar que o índice seja afetado pela diferença de tamanhos de população entre os municípios

Como sabe-se, os dados sobre o tamanho da população são coletados em censos que não ocorrem anualmente. Portanto, para solucionar esse problema utilizaremos a estimativa populacional fornecida pelo IBGE (obtidas através do Sidra). [Estimativas - SIDRA](https://sidra.ibge.gov.br/tabela/6579). Para 2022, utilizamos as [prévias do censo](https://www.ibge.gov.br/estatisticas/sociais/populacao/22827-censo-demografico-2022.html?edicao=35938&t=resultados) divulgadas pelo IBGE

# Bibliotecas utilizadas

In [1]:
import pandas as pd
import numpy as np
from google.colab import files, drive
from functools import reduce

In [2]:
pip install unidecode


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.0 MB/s eta 0:00:00


In [3]:
import unidecode

# Processamento das bases originais

##### As células de código presentes nessa seção não precisam ser rodadas. Estou mantendo-as aqui com o intuito de documentar o que foi feito.



*   Dados criminais
*   Dados IBGE



As tabelas obtidas no [Portal SSP](https://www.ssp.sp.gov.br/estatistica/dados-mensais) são divididas por crimes e por ano - com as ocorrências criminais computadas mensalmente. Dessa maneira, como estamos analisando 12 crimes, para cada ano existem 12 tabelas. Para facilitar a importação e a análise, criamos uma tabela para cada ano que será analisado (2014, 2016, 2018, 2020, 2022)

Nessas novas tabelas, as colunas em comum (listadas no código abaixo) dos dataframes originais foram mantidas. Retiramos as colunas com as notificações mensais e mantivemos a soma anual da ocorrência de crime no município.

```
def processar(ano):
    files = {
        'furto_veiculo': f'/content/furto_de_veículo_{ano}.csv',
        'furto_outros': f'/content/furto_outros_{ano}.csv',
        'homicidio_culposo_outros': f'/content/homicidio_culposo_outros_{ano}.csv',
        'homicidio_culposo_por_acidente': f'/content/homicidio_culposo_por_acidente_de_trânsito_{ano}.csv',
        'latrocinio': f'/content/latrocinio_{ano}.csv',
        'lesao_culposa_outros': f'/content/lesão_corporal_culposa_outras_{ano}.csv',
        'lesao_culposa_acidente': f'/content/lesão_corporal_culposa_por_acidente_de_trânsito_{ano}.csv',
        'lesao_dolosa': f'/content/lesão_corporal_dolosa_{ano}.csv',
        'vitimas_homicidio': f'/content/nº_de_vítimas_em_homicídio_doloso_3_{ano}.csv',
        'tentativa_homicidio': f'/content/tentativa_de_homicídio_{ano}.csv',
        'total_estupro': f'/content/total_de_estupro_4_{ano}.csv',
        'total_roubos': f'/content/total_de_roubo_outros_1_{ano}.csv'
    }

    dataframes = []

    for tipo_crime, file in files.items():
        df = pd.read_csv(file)
        df = df[['Região Geográfica Intermediária', 'Nome Região Geográfica Intermediária',
                  'Região Geográfica Imediata', 'Nome Região Geográfica Imediata',
                  'Total', 'municipios', 'cod_ibge', 'reg_administrativa', 'reg_metropolitanas']]
        df.rename(columns={'Total': f'{tipo_crime.replace("_", " ").capitalize()} {ano}'}, inplace=True)
        dataframes.append(df)

    colunas_comuns = ['Região Geográfica Intermediária', 'Nome Região Geográfica Intermediária',
                      'Região Geográfica Imediata', 'Nome Região Geográfica Imediata',
                      'municipios', 'cod_ibge', 'reg_administrativa', 'reg_metropolitanas']

    df_combined = reduce(lambda left, right: pd.merge(left, right, on=colunas_comuns, how='outer'), dataframes)

    nova_ordem = colunas_comuns + [f'{tipo_crime.replace("_", " ").capitalize()} {ano}' for tipo_crime in files.keys()]
    df_combined = df_combined[nova_ordem]

    return df_combined
```


Abaixo temos um exemplo de como utilizar a função. Apenas um ano foi exposto pois o processo é o mesmo para os demais.
```
df_2014 = processar(2014)
```


Após processamento as bases foram baixadas para facilitar a importação na hora de utilizar os dados. Abaixo tem-se o exemplo de como o download foi feito para o ano de 2014, mas assim como na função exposta acima, foi-se feito para todos os anos.
```
df_2014.to_csv('df_2014.csv', index=False)

files.download('df_2014.csv')
```






In [ ]:
#@title Fução para importar dados e processar as tabelas
#@markdown

def processar(ano):
    files = {
        'furto_veiculo': f'/content/furto_de_veículo_{ano}.csv',
        'furto_outros': f'/content/furto_outros_{ano}.csv',
        'homicidio_culposo_outros': f'/content/homicidio_culposo_outros_{ano}.csv',
        'homicidio_culposo_por_acidente': f'/content/homicidio_culposo_por_acidente_de_trânsito_{ano}.csv',
        'latrocinio': f'/content/latrocinio_{ano}.csv',
        'lesao_culposa_outros': f'/content/lesão_corporal_culposa_outras_{ano}.csv',
        'lesao_culposa_acidente': f'/content/lesão_corporal_culposa_por_acidente_de_trânsito_{ano}.csv',
        'lesao_dolosa': f'/content/lesão_corporal_dolosa_{ano}.csv',
        'vitimas_homicidio': f'/content/nº_de_vítimas_em_homicídio_doloso_3_{ano}.csv',
        'tentativa_homicidio': f'/content/tentativa_de_homicídio_{ano}.csv',
        'total_estupro': f'/content/total_de_estupro_4_{ano}.csv',
        'total_roubos': f'/content/total_de_roubo_outros_1_{ano}.csv'
    }

    dataframes = []

    for tipo_crime, file in files.items():
        df = pd.read_csv(file)
        df = df[['Região Geográfica Intermediária', 'Nome Região Geográfica Intermediária',
                  'Região Geográfica Imediata', 'Nome Região Geográfica Imediata',
                  'Total', 'municipios', 'cod_ibge', 'reg_administrativa', 'reg_metropolitanas']]
        df.rename(columns={'Total': f'{tipo_crime.replace("_", " ").capitalize()} {ano}'}, inplace=True)
        dataframes.append(df)

    colunas_comuns = ['Região Geográfica Intermediária', 'Nome Região Geográfica Intermediária',
                      'Região Geográfica Imediata', 'Nome Região Geográfica Imediata',
                      'municipios', 'cod_ibge', 'reg_administrativa', 'reg_metropolitanas']

    df_combined = reduce(lambda left, right: pd.merge(left, right, on=colunas_comuns, how='outer'), dataframes)

    nova_ordem = colunas_comuns + [f'{tipo_crime.replace("_", " ").capitalize()} {ano}' for tipo_crime in files.keys()]
    df_combined = df_combined[nova_ordem]

    return df_combined


## 2014

In [ ]:
df_2014 = processar(2014)
df_2014.head(3)

,Região Geográfica Intermediária,Nome Região Geográfica Intermediária,Região Geográfica Imediata,Nome Região Geográfica Imediata,municipios,cod_ibge,reg_administrativa,reg_metropolitanas,Furto veiculo 2014,Furto outros 2014,Homicidio culposo outros 2014,Homicidio culposo por acidente 2014,Latrocinio 2014,Lesao culposa outros 2014,Lesao culposa acidente 2014,Lesao dolosa 2014,Vitimas homicidio 2014,Tentativa homicidio 2014,Total estupro 2014,Total roubos 2014
0,3510,Campinas,350038,Campinas,Campinas,3509502,RA de Campinas,RM de Campinas,4282,19326,5,151,8,79,4073,4116,157,129,251,8949
1,3510,Campinas,350038,Campinas,Holambra,3519055,RA de Campinas,RM de Campinas,8,120,0,2,1,0,40,48,2,0,1,15
2,3510,Campinas,350038,Campinas,Indaiatuba,3520509,RA de Campinas,RM de Campinas,317,1602,0,30,0,171,702,605,9,24,40,644


## 2016

In [ ]:
df_2016 = processar(2016)
df_2016.head(3)

,Região Geográfica Intermediária,Nome Região Geográfica Intermediária,Região Geográfica Imediata,Nome Região Geográfica Imediata,municipios,cod_ibge,reg_administrativa,reg_metropolitanas,Furto veiculo 2016,Furto outros 2016,Homicidio culposo outros 2016,Homicidio culposo por acidente 2016,Latrocinio 2016,Lesao culposa outros 2016,Lesao culposa acidente 2016,Lesao dolosa 2016,Vitimas homicidio 2016,Tentativa homicidio 2016,Total estupro 2016,Total roubos 2016
0,3510,Campinas,350038,Campinas,Campinas,3509502,RA de Campinas,RM de Campinas,4027,17440,4,111,6,186,2905,3955,119,91,276,9451
1,3510,Campinas,350038,Campinas,Holambra,3519055,RA de Campinas,RM de Campinas,9,83,0,0,0,1,41,47,0,3,2,14
2,3510,Campinas,350038,Campinas,Indaiatuba,3520509,RA de Campinas,RM de Campinas,222,1743,0,14,0,53,332,703,5,11,58,627


## 2018

In [ ]:
df_2018 = processar(2018)
df_2018.head(3)

,Região Geográfica Intermediária,Nome Região Geográfica Intermediária,Região Geográfica Imediata,Nome Região Geográfica Imediata,municipios,cod_ibge,reg_administrativa,reg_metropolitanas,Furto veiculo 2018,Furto outros 2018,Homicidio culposo outros 2018,Homicidio culposo por acidente 2018,Latrocinio 2018,Lesao culposa outros 2018,Lesao culposa acidente 2018,Lesao dolosa 2018,Vitimas homicidio 2018,Tentativa homicidio 2018,Total estupro 2018,Total roubos 2018
0,3510,Campinas,350038,Campinas,Campinas,3509502,RA de Campinas,RM de Campinas,3849,16282,5,77,5,22,2264,3314,143,69,269,7034
1,3510,Campinas,350038,Campinas,Holambra,3519055,RA de Campinas,RM de Campinas,7,101,0,0,0,1,38,37,0,0,2,7
2,3510,Campinas,350038,Campinas,Indaiatuba,3520509,RA de Campinas,RM de Campinas,123,1759,0,19,0,5,422,708,7,5,83,403


## 2020

In [ ]:
df_2020 = processar(2020)
df_2020.head(3)

,Região Geográfica Intermediária,Nome Região Geográfica Intermediária,Região Geográfica Imediata,Nome Região Geográfica Imediata,municipios,cod_ibge,reg_administrativa,reg_metropolitanas,Furto veiculo 2020,Furto outros 2020,Homicidio culposo outros 2020,Homicidio culposo por acidente 2020,Latrocinio 2020,Lesao culposa outros 2020,Lesao culposa acidente 2020,Lesao dolosa 2020,Vitimas homicidio 2020,Tentativa homicidio 2020,Total estupro 2020,Total roubos 2020
0,3510,Campinas,350038,Campinas,Campinas,3509502,RA de Campinas,RM de Campinas,2702,11401,1,104,10,39,1378,2814,126,69,231,4999
1,3510,Campinas,350038,Campinas,Holambra,3519055,RA de Campinas,RM de Campinas,7,59,0,1,0,0,24,23,1,2,3,3
2,3510,Campinas,350038,Campinas,Indaiatuba,3520509,RA de Campinas,RM de Campinas,172,1424,0,19,1,4,253,428,3,11,65,312


## 2022

In [ ]:
df_2022 = processar(2022)
df_2022.head(3)

,Região Geográfica Intermediária,Nome Região Geográfica Intermediária,Região Geográfica Imediata,Nome Região Geográfica Imediata,municipios,cod_ibge,reg_administrativa,reg_metropolitanas,Furto veiculo 2022,Furto outros 2022,Homicidio culposo outros 2022,Homicidio culposo por acidente 2022,Latrocinio 2022,Lesao culposa outros 2022,Lesao culposa acidente 2022,Lesao dolosa 2022,Vitimas homicidio 2022,Tentativa homicidio 2022,Total estupro 2022,Total roubos 2022
0,3510,Campinas,350038,Campinas,Campinas,3509502,RA de Campinas,RM de Campinas,3510,17252,2,94,5,57,1336,3548,121,74,312,5568
1,3510,Campinas,350038,Campinas,Holambra,3519055,RA de Campinas,RM de Campinas,11,81,0,1,0,1,22,25,0,0,2,3
2,3510,Campinas,350038,Campinas,Indaiatuba,3520509,RA de Campinas,RM de Campinas,170,1799,0,15,0,11,264,726,7,14,71,282


In [ ]:
df_2014.to_csv('df_2014.csv', index=False)
df_2016.to_csv('df_2016.csv', index=False)
df_2018.to_csv('df_2018.csv', index=False)
df_2020.to_csv('df_2020.csv', index=False)
df_2022.to_csv('df_2022.csv', index=False)

files.download('df_2014.csv')
files.download('df_2016.csv')
files.download('df_2018.csv')
files.download('df_2020.csv')
files.download('df_2022.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Importação das bases

In [19]:
from os import path

!git clone https://github.com/MurilloYumoto/Analise-Criminal-no-Brasil

path_EST_POP = '/content/Analise-Criminal-no-Brasil/Processamento_base_EST_POP/EST_POP_processada.csv'

fatal: destination path 'Analise-Criminal-no-Brasil' already exists and is not an empty directory.


In [20]:
path_EST_POP

'/content/Analise-Criminal-no-Brasil/Processamento_base_EST_POP/EST_POP_processada.csv'

In [53]:
df_pop_2 = pd.read_csv(path_EST_POP)

df_pop_2['Municipio'] = df_pop_2['Municipio'].apply(lambda x: unidecode.unidecode(x))

df_pop_2.head(3)

,Unnamed: 0,Ano,cod_ibge,Municipio,populacao,homens,mulheres,razao_sexo,id_media,dens_demog
0,0,2000,3500105,Adamantina,33484.0,16318.0,17166.0,"105,2","33,7","81,3"
1,1,2000,3500204,Adolfo,3680.0,1868.0,1812.0,97,"32,2","17,4"
2,2,2000,3500303,Aguai,28145.0,14109.0,14036.0,"99,5","30,3","59,3"


In [54]:
municipios = ['Aguas de Lindoia', 'Amparo','Atibaia','Bom Jesus dos Perdoes','Braganca Paulista',
    'Cabreuva','Campinas','Campo Limpo Paulista','Holambra','Indaiatuba','Itapira','Itatiba',
    'Itupeva','Jaguariuna','Jarinu','Joanopolis','Jundiai','Lindoia','Louveira','Mogi Guacu',
    'Mogi Mirim','Monte Alegre do Sul','Morungaba','Nazare Paulista','Paulinia','Pedra Bela',
    'Pedreira','Pinhalzinho','Piracaia','Santo Antonio de Posse','Serra Negra','Socorro',
    'Tuiuti','Valinhos','Vargem','Varzea Paulista','Vinhedo','Estiva Gerbi']

anos = [2014, 2016, 2018, 2020, 2022]

df_pop_2 = df_pop_2[df_pop_2['Ano'].isin(anos)]
df_pop_2 = df_pop_2[df_pop_2['Municipio'].isin(municipios)]

df_pop_2.rename(columns={'Municipio': 'municipios'}, inplace=True)

df_pop_2 = df_pop_2[['municipios', 'populacao', 'Ano']]

df_pop_2.head(3)

,municipios,populacao,Ano
9034,Aguas de Lindoia,17623.0,2014
9051,Amparo,66789.0,2014
9077,Atibaia,137520.0,2014


In [58]:
df_pop_2['Ano'] = df_pop_2['Ano'].astype(str)

df_pop_pivot = df_pop_2.pivot(index='municipios', columns='Ano', values='populacao')

df_pop_pivot.columns = [f'Populacao {ano}' for ano in df_pop_pivot.columns]

df_pop_pivot = df_pop_pivot.reset_index()

df_pop_pivot.sort_values(by='municipios', inplace=True)

df_pop_pivot.head()

,municipios,Populacao 2014,Populacao 2016,Populacao 2018,Populacao 2020,Populacao 2022
0,Aguas de Lindoia,17623.0,17753.0,17866.0,17960.0,17925.0
1,Amparo,66789.0,67300.0,67691.0,68032.0,67993.0
2,Atibaia,137520.0,143086.0,148586.0,153965.0,158399.0
3,Bom Jesus dos Perdoes,20407.0,20859.0,21299.0,21693.0,21989.0
4,Braganca Paulista,156920.0,162244.0,167473.0,172439.0,176582.0


In [4]:
# bases com ocorrências criminais
df_2014 = pd.read_csv('/content/df_2014.csv')
df_2016 = pd.read_csv('/content/df_2016.csv')
df_2018 = pd.read_csv('/content/df_2018.csv')
df_2020 = pd.read_csv('/content/df_2020.csv')
df_2022 = pd.read_csv('/content/df_2022.csv')

# bases com estimativas populacionais
df_pop = pd.read_csv('/content/Estimativas_pop.csv')

In [5]:
#removendo acentos da coluna municípios e ordenando em ordem alfabética
df_2014['municipios'] = df_2014['municipios'].apply(lambda x: unidecode.unidecode(x))
df_2014 = df_2014.sort_values(by='municipios')

df_2016['municipios'] = df_2016['municipios'].apply(lambda x: unidecode.unidecode(x))
df_2016 = df_2016.sort_values(by='municipios')

df_2018['municipios'] = df_2018['municipios'].apply(lambda x: unidecode.unidecode(x))
df_2018 = df_2018.sort_values(by='municipios')

df_2020['municipios'] = df_2020['municipios'].apply(lambda x: unidecode.unidecode(x))
df_2020 = df_2020.sort_values(by='municipios')

df_2022['municipios'] = df_2022['municipios'].apply(lambda x: unidecode.unidecode(x))
df_2022 = df_2022.sort_values(by='municipios')

In [6]:
df_pop['Município'] = df_pop['Município'].str.replace(' \(SP\)', '', regex=True)

df_pop.rename(columns={'Município': 'municipios'}, inplace=True)

df_pop['municipios'] = df_pop['municipios'].apply(lambda x: unidecode.unidecode(x))

df_pop.sort_values(by='municipios', inplace=True)

df_pop.rename(columns={'2014': 'Populacao 2014', '2016': 'Populacao 2016', '2018': 'Populacao 2018', '2020': 'Populacao 2020', '2022': 'Populacao 2022'}, inplace=True)

df_pop.head(3)

,municipios,Populacao 2014,Populacao 2016,Populacao 2018,Populacao 2020,Populacao 2022
0,Aguas de Lindoia,18212,18412,18599,18808,18859
1,Amparo,69808,70742,71700,72677,69952
2,Atibaia,135895,138449,141398,144088,171672


In [17]:
df_pop

,municipios,Populacao 2014,Populacao 2016,Populacao 2018,Populacao 2020,Populacao 2022
0,Aguas de Lindoia,18212,18412,18599,18808,18859
1,Amparo,69808,70742,71700,72677,69952
2,Atibaia,135895,138449,141398,144088,171672
3,Bom Jesus dos Perdoes,22508,23530,24898,25985,24365
4,Braganca Paulista,158856,162435,166753,170533,181556
5,Cabreuva,45830,47210,48966,50429,45941
6,Campinas,1154617,1173370,1194094,1213792,1170247
7,Campo Limpo Paulista,79982,81693,83735,85541,81355
37,Estiva Gerbi,10772,10971,11198,11407,11394
8,Holambra,13046,13698,14579,15272,16194


In [7]:
df_2014_pop = pd.merge(df_2014, df_pop[['municipios', 'Populacao 2014']], on='municipios', how='left')
df_2016_pop = pd.merge(df_2016, df_pop[['municipios', 'Populacao 2016']], on='municipios', how='left')
df_2018_pop = pd.merge(df_2018, df_pop[['municipios', 'Populacao 2018']], on='municipios', how='left')
df_2020_pop = pd.merge(df_2020, df_pop[['municipios', 'Populacao 2020']], on='municipios', how='left')
df_2022_pop = pd.merge(df_2022, df_pop[['municipios', 'Populacao 2022']], on='municipios', how='left')

In [61]:
df_2014_pop_2 = pd.merge(df_2014, df_pop_pivot[['municipios', 'Populacao 2014']], on='municipios', how='left')
df_2016_pop_2 = pd.merge(df_2016, df_pop_pivot[['municipios', 'Populacao 2016']], on='municipios', how='left')
df_2018_pop_2 = pd.merge(df_2018, df_pop_pivot[['municipios', 'Populacao 2018']], on='municipios', how='left')
df_2020_pop_2 = pd.merge(df_2020, df_pop_pivot[['municipios', 'Populacao 2020']], on='municipios', how='left')
df_2022_pop_2 = pd.merge(df_2022, df_pop_pivot[['municipios', 'Populacao 2022']], on='municipios', how='left')

# Cálculo do índice

## Definição dos pesos e da fórmula para cálculo

In [8]:
pesos = {
    'Furto veiculo': 1,
    'Furto outros': 1,
    'Homicidio culposo outros': 4,
    'Homicidio culposo por acidente': 4.5,
    'Latrocinio': 6,
    'Lesao culposa outros': 2.5,
    'Lesao culposa acidente': 3,
    'Lesao dolosa': 3.5,
    'Vitimas homicidio': 5,
    'Tentativa homicidio': 4.5,
    'Total estupro': 5,
    'Total roubos': 2
}

In [9]:
def calcular_indice_criminalidade(row, ano):
    numerador = sum((row[f"{crime} {ano}"] / row[f'Populacao {ano}']) * peso
                    for crime, peso in pesos.items() if f"{crime} {ano}" in row)
    denominador = sum(peso for crime, peso in pesos.items() if f"{crime} {ano}" in row)
    return numerador / denominador if denominador > 0 else 0

## Execução dados SIDRA



In [10]:
anos = ['2014', '2016', '2018', '2020', '2022']
dataframes = [df_2014_pop, df_2016_pop, df_2018_pop, df_2020_pop, df_2022_pop]

for df, ano in zip(dataframes, anos):
    df['Indice Criminalidade'] = df.apply(lambda row: calcular_indice_criminalidade(row, ano), axis=1)

In [11]:
df_index_2014 = df_2014_pop[['cod_ibge', 'municipios', 'Indice Criminalidade']].rename(columns={'Indice Criminalidade': 'Indice Criminalidade 2014'})
df_index_2016 = df_2016_pop[['cod_ibge', 'municipios', 'Indice Criminalidade']].rename(columns={'Indice Criminalidade': 'Indice Criminalidade 2016'})
df_index_2018 = df_2018_pop[['cod_ibge', 'municipios', 'Indice Criminalidade']].rename(columns={'Indice Criminalidade': 'Indice Criminalidade 2018'})
df_index_2020 = df_2020_pop[['cod_ibge', 'municipios', 'Indice Criminalidade']].rename(columns={'Indice Criminalidade': 'Indice Criminalidade 2020'})
df_index_2022 = df_2022_pop[['cod_ibge', 'municipios', 'Indice Criminalidade']].rename(columns={'Indice Criminalidade': 'Indice Criminalidade 2022'})

df_final = pd.concat([df_index_2014, df_index_2016['Indice Criminalidade 2016'],
                      df_index_2018['Indice Criminalidade 2018'],
                      df_index_2020['Indice Criminalidade 2020'],
                      df_index_2022['Indice Criminalidade 2022']], axis=1)

In [12]:
#normalização dos dados
def normalizar_serie(serie):
    min_value = serie.min()
    max_value = serie.max()
    return (serie - min_value) / (max_value - min_value) if (max_value - min_value) > 0 else serie

for ano in anos:
    coluna = f'Indice Criminalidade {ano}'
    df_final[f'Indice Criminalidade Normalizado {ano}'] = normalizar_serie(df_final[coluna])

## Resultado

In [13]:
df_normalizado = df_final[['cod_ibge', 'municipios', 'Indice Criminalidade Normalizado 2014', 'Indice Criminalidade Normalizado 2016', 'Indice Criminalidade Normalizado 2018',
                           'Indice Criminalidade Normalizado 2020', 'Indice Criminalidade Normalizado 2022']]

df_normalizado.head(3)

,cod_ibge,municipios,Indice Criminalidade Normalizado 2014,Indice Criminalidade Normalizado 2016,Indice Criminalidade Normalizado 2018,Indice Criminalidade Normalizado 2020,Indice Criminalidade Normalizado 2022
0,3500501,Aguas de Lindoia,0.574364,0.302664,0.463143,0.010731,0.120750
1,3501905,Amparo,0.461050,0.532752,0.476568,0.000000,0.149246
2,3504107,Atibaia,0.912955,1.000000,0.757941,0.710423,0.312955


## Classificação

In [14]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
labels = ['Muito Baixo', 'Baixo', 'Médio', 'Alto', 'Muito Alto']

df_normalizado = df_normalizado.copy()

for ano in anos:
    coluna_normalizada = f'Indice Criminalidade Normalizado {ano}'
    coluna_classificacao = f'Classificacao {ano}'
    df_normalizado[coluna_classificacao] = pd.cut(df_final[coluna_normalizada], bins=bins, labels=labels, include_lowest=True)

df_normalizado = df_normalizado[['cod_ibge', 'municipios', 'Indice Criminalidade Normalizado 2014', 'Classificacao 2014', 'Indice Criminalidade Normalizado 2016',
                                 'Classificacao 2016', 'Indice Criminalidade Normalizado 2018', 'Classificacao 2018', 'Indice Criminalidade Normalizado 2020',
                                 'Classificacao 2020', 'Indice Criminalidade Normalizado 2022', 'Classificacao 2022']]
df_normalizado.head(3)

,cod_ibge,municipios,Indice Criminalidade Normalizado 2014,Classificacao 2014,Indice Criminalidade Normalizado 2016,Classificacao 2016,Indice Criminalidade Normalizado 2018,Classificacao 2018,Indice Criminalidade Normalizado 2020,Classificacao 2020,Indice Criminalidade Normalizado 2022,Classificacao 2022
0,3500501,Aguas de Lindoia,0.574364,Médio,0.302664,Baixo,0.463143,Médio,0.010731,Muito Baixo,0.120750,Muito Baixo
1,3501905,Amparo,0.461050,Médio,0.532752,Médio,0.476568,Médio,0.000000,Muito Baixo,0.149246,Muito Baixo
2,3504107,Atibaia,0.912955,Muito Alto,1.000000,Muito Alto,0.757941,Alto,0.710423,Alto,0.312955,Baixo


In [16]:
#@title baixando df
df_normalizado

df_normalizado.to_csv('df_normalizado.csv', index=False)

files.download('df_normalizado.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Execução dados SEADE

In [63]:
dataframes = [df_2014_pop_2, df_2016_pop_2, df_2018_pop_2, df_2020_pop_2, df_2022_pop_2]

for df, ano in zip(dataframes, anos):
    df['Indice Criminalidade'] = df.apply(lambda row: calcular_indice_criminalidade(row, ano), axis=1)

In [64]:
df_index_2014_2 = df_2014_pop_2[['cod_ibge', 'municipios', 'Indice Criminalidade']].rename(columns={'Indice Criminalidade': 'Indice Criminalidade 2014'})
df_index_2016_2 = df_2016_pop_2[['cod_ibge', 'municipios', 'Indice Criminalidade']].rename(columns={'Indice Criminalidade': 'Indice Criminalidade 2016'})
df_index_2018_2 = df_2018_pop_2[['cod_ibge', 'municipios', 'Indice Criminalidade']].rename(columns={'Indice Criminalidade': 'Indice Criminalidade 2018'})
df_index_2020_2 = df_2020_pop_2[['cod_ibge', 'municipios', 'Indice Criminalidade']].rename(columns={'Indice Criminalidade': 'Indice Criminalidade 2020'})
df_index_2022_2 = df_2022_pop_2[['cod_ibge', 'municipios', 'Indice Criminalidade']].rename(columns={'Indice Criminalidade': 'Indice Criminalidade 2022'})

df_final_2 = pd.concat([df_index_2014_2, df_index_2016_2['Indice Criminalidade 2016'],
                      df_index_2018_2['Indice Criminalidade 2018'],
                      df_index_2020_2['Indice Criminalidade 2020'],
                      df_index_2022_2['Indice Criminalidade 2022']], axis=1)

In [66]:
for ano in anos:
    coluna = f'Indice Criminalidade {ano}'
    df_final_2[f'Indice Criminalidade Normalizado {ano}'] = normalizar_serie(df_final_2[coluna])

In [67]:
df_normalizado_2 = df_final_2[['cod_ibge', 'municipios', 'Indice Criminalidade Normalizado 2014', 'Indice Criminalidade Normalizado 2016', 'Indice Criminalidade Normalizado 2018',
                           'Indice Criminalidade Normalizado 2020', 'Indice Criminalidade Normalizado 2022']]

df_normalizado_2.head(3)

,cod_ibge,municipios,Indice Criminalidade Normalizado 2014,Indice Criminalidade Normalizado 2016,Indice Criminalidade Normalizado 2018,Indice Criminalidade Normalizado 2020,Indice Criminalidade Normalizado 2022
0,3500501,Aguas de Lindoia,0.563033,0.335446,0.480963,0.013998,0.113259
1,3501905,Amparo,0.465091,0.580210,0.513114,0.022164,0.125659
2,3504107,Atibaia,0.825387,0.916372,0.640838,0.504822,0.344794


In [68]:
#@title 2
df_normalizado_2 = df_normalizado_2.copy()

for ano in anos:
    coluna_normalizada = f'Indice Criminalidade Normalizado {ano}'
    coluna_classificacao = f'Classificacao {ano}'
    df_normalizado_2[coluna_classificacao] = pd.cut(df_final_2[coluna_normalizada], bins=bins, labels=labels, include_lowest=True)

df_normalizado_2 = df_normalizado_2[['cod_ibge', 'municipios', 'Indice Criminalidade Normalizado 2014', 'Classificacao 2014', 'Indice Criminalidade Normalizado 2016',
                                 'Classificacao 2016', 'Indice Criminalidade Normalizado 2018', 'Classificacao 2018', 'Indice Criminalidade Normalizado 2020',
                                 'Classificacao 2020', 'Indice Criminalidade Normalizado 2022', 'Classificacao 2022']]
df_normalizado_2.head(3)

,cod_ibge,municipios,Indice Criminalidade Normalizado 2014,Classificacao 2014,Indice Criminalidade Normalizado 2016,Classificacao 2016,Indice Criminalidade Normalizado 2018,Classificacao 2018,Indice Criminalidade Normalizado 2020,Classificacao 2020,Indice Criminalidade Normalizado 2022,Classificacao 2022
0,3500501,Aguas de Lindoia,0.563033,Médio,0.335446,Baixo,0.480963,Médio,0.013998,Muito Baixo,0.113259,Muito Baixo
1,3501905,Amparo,0.465091,Médio,0.580210,Médio,0.513114,Médio,0.022164,Muito Baixo,0.125659,Muito Baixo
2,3504107,Atibaia,0.825387,Muito Alto,0.916372,Muito Alto,0.640838,Alto,0.504822,Médio,0.344794,Baixo


In [69]:
df_normalizado_2.to_csv('df_normalizado_2.csv', index=False)

files.download('df_normalizado_2.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>